# NavWareSet Quantitative Analysis

This notebook is a quantitative analysis of 545 hand selected robot-human tracks from the NavWareSet dataset. Almost all tracks from all scenarios where compliant and non-compliant behavior are present are analysed. All metrics were calculated for each possible combination of scenario, robot and behavior. Our objective is not to propose new metrics, but to test how our new dataset scores against metrics already proposed in the literature about robot social navigation. 

The chosen metrics were:
- Vavg (m/s) [1] - Average velocity of the robot across a number of tracks;
- Aavg (m/s2) [1] - Average aceleration of the robot across a number of tracks;
- Javg (m/s3) [1] - Average Jerk of the robot across a number of tracks;
- PLavg (m) [1] - Agerage Path Length across a number of tracks;
- PSC (%) [2] - Ratio of the trajectory with the minimum distance to a human under a given threshold. If the threshold is 0.5m, it is referred to as Personal Space Compliance (PSC);
- DHmin [3] - Average minimum distance to a human across a number of tracks;

[(1) - Arena-Bench: A Benchmarking Suite for Obstacle Avoidance Approaches in Highly Dynamic Environments](https://arxiv.org/pdf/2206.05728)

[(2) - iGibson 2.0: Object-Centric Simulation for Robot Learning of Everyday Household Tasks](https://arxiv.org/pdf/2108.03272)

[(3) - Principles and Guidelines for Evaluating Social Robot Navigation Algorithms](https://arxiv.org/pdf/2306.16740)

## Separating the scenes

In total, 5 scenes were analysed:
- Frontal Approach (fa);
- Pedestrian Obstruction (po);
- Blind Corner (bc);
- Perpendicular Crossing (pc);
- Circular Crossing (cc).

Two robots were used:
- Cleapath Jackal (j);
- Toyota HSR (h).

And two robot behaviors were observed:
- Compliant (c);
- Non-compliant (n).

In the cell below all scenes are divided acording to the classifications above.

In [10]:
# scenes corresponding to each scenario combination
fa_h_c = [1, 27]
fa_h_n = [8, 34]
fa_j_c = [15, 41]
fa_j_n = [21, 47]
po_h_c = [2, 28]
po_h_n = [9, 35]
po_j_c = [16, 42]
po_j_n = [22]
bc_h_c = [3, 29]
bc_h_n = [10, 36]
bc_j_c = [17, 43]
bc_j_n = [23, 49]
pc_h_c = [5, 31]
pc_h_n = [12, 38]
pc_j_c = [19, 45]
pc_j_n = [25, 51]
cc_h_c = [6, 32]
cc_h_n = [13, 39]
cc_j_c = [20, 46]
cc_j_n = [26, 52]

## Calculating the metrics

Down below we see the functions implemented to calculate each of the forementioned metrics

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

root = Path("selected_tracks")

def global_average_speed(scene_list: list) -> float:
    """
    Global average robot speed across ALL tracks in ALL scenes (m/s), computed as:
        (sum of distances over all tracks) / (sum of durations over all tracks)

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    total_dist = 0.0
    total_time = 0.0

    for scene_id, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            t = df["timestamp"].to_numpy(dtype=np.float64) * 1e-9  # seconds
            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2).sum()
            dt = t[-1] - t[0]

            if dt > 0:
                total_dist += dist
                total_time += dt

    return total_dist / total_time if total_time > 0 else 0.0

print(global_average_speed(po_j_n))


0.666501013767755


In [25]:
def global_average_acceleration(scene_list: list) -> float:
    """
    Global average robot acceleration magnitude across ALL tracks in ALL scenes (m/s^2).

    For each track:
      1) compute instantaneous speed v_i = ds_i / dt_i
      2) compute instantaneous acceleration a_i = dv_i / dt_i
      3) take mean(|a_i|) over the track (time-weighted)
    Then average globally across all tracks (time-weighted).

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    sum_a_dt = 0.0
    sum_dt = 0.0

    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            t = df["timestamp"].to_numpy(dtype=np.float64) * 1e-9  # seconds
            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            dt = np.diff(t)
            dx = np.diff(x)
            dy = np.diff(y)

            valid = dt > 0
            dt = dt[valid]
            dx = dx[valid]
            dy = dy[valid]

            if len(dt) < 2:
                continue

            # speed per interval
            ds = np.sqrt(dx**2 + dy**2)
            v = ds / dt  # m/s

            # acceleration per interval between speeds
            dv = np.diff(v)
            dt2 = dt[1:]  # time between v samples
            valid2 = dt2 > 0

            a = np.abs(dv[valid2] / dt2[valid2])  # m/s^2

            if len(a) == 0:
                continue

            # time-weighted accumulation for global mean
            sum_a_dt += np.sum(a * dt2[valid2])
            sum_dt += np.sum(dt2[valid2])

    return sum_a_dt / sum_dt if sum_dt > 0 else 0.0

print(global_average_acceleration(po_j_n))


0.5595769171212424


In [26]:
def global_average_jerk(scene_list: list) -> float:
    """
    Global average robot jerk magnitude across ALL tracks in ALL scenes (m/s^3).

    For each track:
      1) v_i = ds_i / dt_i
      2) a_i = dv_i / dt_i
      3) j_i = da_i / dt_i
      4) average |j_i| using a time-weighted mean
    Then average globally across all tracks (time-weighted).

    scene_list format example: [22] or [1, 27] or any list of scene IDs
    """
    files_by_scene = {s: list(root.rglob(f"*scene{s}_*.csv")) for s in scene_list}
    
    sum_j_dt = 0.0
    sum_dt = 0.0

    for _, track_list in files_by_scene.items():
        for p in track_list:
            df = pd.read_csv(p)

            t = df["timestamp"].to_numpy(dtype=np.float64) * 1e-9  # seconds
            x = df["robot_x"].to_numpy(dtype=np.float64)
            y = df["robot_y"].to_numpy(dtype=np.float64)

            dt = np.diff(t)
            dx = np.diff(x)
            dy = np.diff(y)

            valid = dt > 0
            dt = dt[valid]
            dx = dx[valid]
            dy = dy[valid]

            if len(dt) < 3:
                continue

            # v_i defined over dt[i]
            ds = np.sqrt(dx**2 + dy**2)
            v = ds / dt  # m/s

            # a_i defined over dt[1:]
            dv = np.diff(v)
            dt_a = dt[1:]
            valid_a = dt_a > 0
            a = dv[valid_a] / dt_a[valid_a]  # signed accel

            if len(a) < 2:
                continue

            # j_i defined over dt_a[1:]
            da = np.diff(a)
            dt_j = dt_a[1:][valid_a[1:]]  # keep consistent indexing
            valid_j = dt_j > 0

            j = np.abs(da[valid_j] / dt_j[valid_j])  # m/s^3

            if len(j) == 0:
                continue

            # time-weighted global mean of |jerk|
            sum_j_dt += np.sum(j * dt_j[valid_j])
            sum_dt += np.sum(dt_j[valid_j])

    return sum_j_dt / sum_dt if sum_dt > 0 else 0.0

print(global_average_jerk(po_j_n))


8.443075226144948
